**Задание**
#### На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN  и (RNN -> CNN)
4. сдлать выводы что получилось лучше


In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd


In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [4]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [5]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re



In [13]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 

In [10]:
# Предобработка
from string import punctuation
from stop_words import get_stop_words
from nltk.stem import SnowballStemmer

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
stemmer = SnowballStemmer("russian")

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [stemmer.stem(word) for word in txt.split() if word not in sw]
    return " ".join(txt)

In [11]:
df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [15]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [16]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [17]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [22]:
from keras.callbacks import EarlyStopping  

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, LSTM, Dense, concatenate, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from keras.layers import Dropout

def build_cnn_model(input_length, vocab_size):
    inputs = Input(shape=(input_length,))
    embedding = Embedding(vocab_size, 100)(inputs)
    conv_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding)
    maxpool_layer = MaxPooling1D(pool_size=2)(conv_layer)
    dropout_layer = Dropout(0.5)(maxpool_layer)  # Добавляем Dropout
    global_maxpool = GlobalMaxPooling1D()(dropout_layer)
    output_layer = Dense(1, activation='sigmoid')(global_maxpool)

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

def build_rnn_model(input_length, vocab_size):
    inputs = Input(shape=(input_length,))
    embedding = Embedding(vocab_size, 100)(inputs)
    lstm_layer = LSTM(128, dropout=0.5, recurrent_dropout=0.5)(embedding)  # Добавляем Dropout
    output_layer = Dense(1, activation='sigmoid')(lstm_layer)

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

def build_cnn_rnn_model(input_length, vocab_size):
    inputs = Input(shape=(input_length,))
    embedding = Embedding(vocab_size, 100)(inputs)
    conv_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding)
    maxpool_layer = MaxPooling1D(pool_size=2)(conv_layer)
    lstm_layer = LSTM(128, dropout=0.5, recurrent_dropout=0.5)(maxpool_layer)  # Добавляем Dropout
    output_layer = Dense(1, activation='sigmoid')(lstm_layer)

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

def build_rnn_cnn_model(input_length, vocab_size):
    inputs = Input(shape=(input_length,))
    embedding = Embedding(vocab_size, 100)(inputs)
    lstm_layer = LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5)(embedding)  # Добавляем Dropout
    conv_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(lstm_layer)
    global_maxpool = GlobalMaxPooling1D()(conv_layer)
    output_layer = Dense(1, activation='sigmoid')(global_maxpool)

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model




In [26]:
# Проверим значения индексов в X_train и X_valid
print(np.max(X_train))
print(np.max(X_valid))

247859
247838


In [27]:
# Установим vocab_size
vocab_size = np.max(np.concatenate([X_train, X_valid])) + 1

print("Vocab Size:", vocab_size)

# Предположим, что maxlen - это максимальная длина последовательности
maxlen = X_train.shape[1]


early_stopping=EarlyStopping(monitor='val_loss')  


# код для создания модели
cnn_model = build_cnn_model(maxlen, vocab_size)
cnn_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

rnn_model = build_rnn_model(maxlen, vocab_size)
rnn_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

cnn_rnn_model = build_cnn_rnn_model(maxlen, vocab_size)
cnn_rnn_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

rnn_cnn_model = build_rnn_cnn_model(maxlen, vocab_size)
rnn_cnn_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])



Vocab Size: 247860
Epoch 1/10
319/319 [==============================] - 17s 52ms/step - loss: 0.5862 - accuracy: 0.6796 - val_loss: 0.5464 - val_accuracy: 0.7247
Epoch 2/10
319/319 [==============================] - 16s 50ms/step - loss: 0.3681 - accuracy: 0.8404 - val_loss: 0.5560 - val_accuracy: 0.7147
Epoch 1/10
319/319 [==============================] - 85s 263ms/step - loss: 0.5495 - accuracy: 0.7078 - val_loss: 0.4924 - val_accuracy: 0.7536
Epoch 2/10
319/319 [==============================] - 87s 272ms/step - loss: 0.3411 - accuracy: 0.8536 - val_loss: 0.5149 - val_accuracy: 0.7505
Epoch 1/10
319/319 [==============================] - 49s 152ms/step - loss: 0.5812 - accuracy: 0.6835 - val_loss: 0.5284 - val_accuracy: 0.7326
Epoch 2/10
319/319 [==============================] - 50s 156ms/step - loss: 0.3422 - accuracy: 0.8515 - val_loss: 0.5914 - val_accuracy: 0.7167
Epoch 1/10
319/319 [==============================] - 60s 186ms/step - loss: 0.5591 - accuracy: 0.6975 - val_loss

In [28]:
score = cnn_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n cnn_model')
print('Test score cnn_model:', score[0])
print('Test accuracy cnn_model:', score[1])

score = rnn_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n rnn_model')
print('Test score rnn_model:', score[0])
print('Test accuracy rnn_model:', score[1])

score = cnn_rnn_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n cnn_rnn_model')
print('Test score cnn_rnn_model:', score[0])
print('Test accuracy cnn_rnn_model:', score[1])

score = rnn_cnn_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n rnn_cnn_model')
print('Test score rnn_cnn_model:', score[0])
print('Test accuracy rnn_cnn_model:', score[1])


45/45 [==============================] - 0s 6ms/step - loss: 0.5847 - accuracy: 0.6971

 cnn_model
Test score cnn_model: 0.5846816301345825
Test accuracy cnn_model: 0.6970859169960022
45/45 [==============================] - 2s 52ms/step - loss: 0.5519 - accuracy: 0.7408

 rnn_model
Test score rnn_model: 0.5518957376480103
Test accuracy rnn_model: 0.7407750487327576
45/45 [==============================] - 1s 30ms/step - loss: 0.6403 - accuracy: 0.6981

 cnn_rnn_model
Test score cnn_rnn_model: 0.6403089165687561
Test accuracy cnn_rnn_model: 0.6980558037757874
45/45 [==============================] - 2s 37ms/step - loss: 0.5450 - accuracy: 0.7404

 rnn_cnn_model
Test score rnn_cnn_model: 0.5449795722961426
Test accuracy rnn_cnn_model: 0.7403782606124878


### ВЫВОДЫ:
- Сравнивая значения Test accuracy и Test loss, можно сказать, что модели rnn_model и rnn_cnn_model показывают лучшие результаты по сравнению с cnn_model и cnn_rnn_model.

- Таким образом, модели, которые включают слои рекуррентной нейронной сети (RNN), rnn_model и rnn_cnn_model, имеют лучшую обобщающую способность на тестовых данных для задачи сентимент анализа по сравнению с моделями cnn_model и cnn_rnn_model.

- Модель cnn_rnn_model содержит сверточный слой (Conv1D), который применяется перед слоем LSTM. Сверточные слои обладают свойством извлекать локальные признаки, в то время как рекуррентные слои (LSTM) способны улавливать долгосрочные зависимости в последовательностях. Уловить долгосрочные зависимости после свертки может быть достаточно сложно.